In [1]:
import os
import sys
import re
import time

project_root = "/root/work/tenset"
os.environ["TVM_HOME"] = f"{project_root}"
os.environ["TVM_LIBRARY_PATH"] = f"{project_root}/build"
if f"{project_root}/python" not in sys.path:
    sys.path.insert(0, f"{project_root}/python")
    

sys.path = [p for p in sys.path if not p.startswith(f"{project_root}/build")]
sys.path.append(f"{project_root}/build")
os.environ["LD_LIBRARY_PATH"] = f"{project_root}/build:" + os.environ.get("LD_LIBRARY_PATH", "")

1. Total 저장

In [1]:
from glob import glob
import pandas as pd



csv_dir = glob(
    "/root/work/tenset/scripts/pre_experiments/model_myself/result/"
    "([[]0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960[]],cuda).json/**/*.csv", recursive=True
)

csvs = []
for csv in csv_dir:
    if "avg" in csv or "total" in csv or "sampling" in csv or "prev" in csv:
       continue
    csvs.append(csv) 

dfs = []
for p in csvs:
    sub_df = pd.read_csv(p)

    if "rank_warmup_epochs" not in sub_df.columns:
        sub_df["rank_warmup_epochs"] = 0
    if "measure_size" not in sub_df.columns:
        sub_df["measure_size"] = 64
    if "scratch" not in sub_df.columns:
        sub_df["scratch"] = False
    if "encoder_freeze" not in sub_df.columns:
        sub_df["encoder_freeze"] = False
    
    dfs.append(sub_df)

    
df_total = pd.concat(dfs, ignore_index=True)

# measure_size 컬럼을 맨 앞으로 이동
cols = df_total.columns.tolist()
df_total = df_total[["measure_size"] + [c for c in cols if c != "measure_size"]]

df_total.to_csv("/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/vae_extent_total.csv", index=True)
df_total

,measure_size,scratch,encoder_freeze,encoder_lr,cost_predictor_lr,weights,uncertainty_topk,grad_num,rand_num,phase,used_time,train_size,val_reg_r2,val_rank_r2,sampling_seed,rank_warmup_epochs,top-1,T_mc
0,64,False,False,0.00010,0.01000,"(0.4, 0.3, 0.3)",64,4,0,1,3.44,64,[0.8153],[None],2000,0,NaN,NaN
1,64,False,False,0.00010,0.01000,"(0.4, 0.3, 0.3)",64,4,0,1,3.44,64,[0.713],[None],2001,0,NaN,NaN
2,64,False,False,0.00010,0.01000,"(0.4, 0.3, 0.3)",64,4,0,1,3.41,64,[0.7737],[None],2002,0,NaN,NaN
3,64,False,False,0.00010,0.01000,"(0.4, 0.3, 0.3)",64,4,0,3,11.19,192,"[0.7897, 0.7438, 0.8066]","[None, None, None]",2003,0,NaN,NaN
4,64,False,False,0.00010,0.01000,"(0.4, 0.3, 0.3)",64,4,0,3,10.98,192,"[0.7887, 0.8399, 0.8845]","[None, None, None]",2004,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,48,False,False,0.00001,0.00001,"(0.4, 0.3, 0.3)",64,4,0,1,3.10,32,[0.6055],[None],2015,500,0.0,NaN
7998,48,False,False,0.00001,0.00001,"(0.4, 0.3, 0.3)",64,4,0,2,6.39,80,"[0.512, 0.3426]","[None, None]",2016,500,1.0,NaN
7999,48,False,False,0.00001,0.00001,"(0.4, 0.3, 0.3)",64,4,0,1,3.10,32,[0.4985],[None],2017,500,0.0,NaN
8000,48,False,False,0.00001,0.00001,"(0.4, 0.3, 0.3)",64,4,0,3,9.81,128,"[0.4944, 0.4233, 0.3679]","[None, None, None]",2018,500,0.0,NaN


2. Total avg 저장

In [2]:
agg_kwargs = {
    "phase": ("phase", "mean"),
    "train_size": ("train_size", "mean"),
    "used_time": ("used_time", "mean"),
    "val_reg_r2": ("val_reg_r2", "first"),
    "seed_n": ("sampling_seed", "nunique"),
    "sampling_seed": ("sampling_seed", list),
}

topk_col = f"top-1"
if topk_col in df_total.columns:
    agg_kwargs[topk_col] = (topk_col, "mean")

group_cols = [
    "measure_size",
    "scratch",
    "encoder_freeze",
    "encoder_lr",
    "cost_predictor_lr",
    "rank_warmup_epochs",
    "weights",
    "uncertainty_topk",
    "grad_num",
    "rand_num",
]

df_total_avg = (
    df_total
    .groupby(group_cols, as_index=False)
    .agg(**agg_kwargs)
)
df_total_avg.to_csv("/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/vae_extent_total_avg.csv", index=True)
df_total_avg

,measure_size,scratch,encoder_freeze,encoder_lr,cost_predictor_lr,rank_warmup_epochs,weights,uncertainty_topk,grad_num,rand_num,phase,train_size,used_time,val_reg_r2,seed_n,sampling_seed,top-1
0,32,False,False,0.00001,0.00001,100,"(0.4, 0.3, 0.3)",0,2,0,6.75,216.0,44.547500,[0.4545],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.150000
1,32,False,False,0.00001,0.00001,100,"(0.4, 0.3, 0.3)",0,4,0,5.80,185.6,31.561500,[0.4545],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.200000
2,32,False,False,0.00001,0.00001,100,"(0.4, 0.3, 0.3)",64,2,0,4.00,128.0,17.559444,"[0.3615, 0.1561, 0.1386, 0.1755, 0.2463, 0.257...",18,"[2002, 2003, 2004, 2005, 2006, 2007, 2008, 200...",0.166667
3,32,False,False,0.00001,0.00001,100,"(0.4, 0.3, 0.3)",64,4,0,4.55,145.6,23.844500,[0.4545],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.250000
4,32,False,False,0.00001,0.00001,200,"(0.4, 0.3, 0.3)",0,2,0,5.50,176.0,29.047500,[0.4735],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,64,False,False,0.00010,0.00010,300,"(0.4, 0.3, 0.3)",64,4,0,1.40,89.6,5.079500,[0.71],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.250000
392,64,False,False,0.00010,0.00010,400,"(0.4, 0.3, 0.3)",64,4,0,1.45,92.8,5.442500,[0.7098],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.250000
393,64,False,False,0.00010,0.00010,500,"(0.4, 0.3, 0.3)",64,4,0,1.55,99.2,5.723000,[0.7091],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",0.300000
394,64,False,False,0.00010,0.00100,0,"(0.4, 0.3, 0.3)",64,4,0,1.60,102.4,5.653000,[0.7838],20,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...",NaN


선택 파일 Avg 저장

In [3]:
def save_avg_csv(filename, top_k):
    df_results = pd.read_csv(filename)
    group_cols = [
        "measure_size",
        "scratch",
        "encoder_freeze",
        "encoder_lr",
        "cost_predictor_lr",
        "rank_warmup_epochs",
        "weights",
        "uncertainty_topk",
        "grad_num",
        "rand_num",
    ]

    df_avg = (
        df_results
        .groupby(group_cols, as_index=False)
        .agg(
            phase=("phase", "mean"),
            train_size=("train_size", "mean"),
            used_time=("used_time", "mean"),
            **{f"top-{top_k}": (f"top-{top_k}", "mean")},
            val_reg_r2=("val_reg_r2", "first"),
            val_rank_r2=("val_rank_r2", "first"),
            seed_n=("sampling_seed", "nunique"),
            sampling_seed=("sampling_seed", list),
        )
    )

    df_avg.to_csv(filename.replace(".csv", "_avg.csv"), index=False)
    df_avg


save_avg_csv("/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/vae_extent_1221_1646.csv")

TypeError: save_avg_csv() missing 1 required positional argument: 'top_k'

두 csv 합치기

In [ ]:
import pandas as pd


# CSV 파일 읽기
csv1 = pd.read_csv('/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/hyper/vae_extent_1220_2013.csv')
csv2 = pd.read_csv('/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/hyper/vae_extent_1220_2135.csv')

# 두 CSV 합치기
merged_csv = pd.concat([csv1, csv2])

# 합친 CSV 저장
merged_csv.to_csv('/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/hyper/vae_extent_hyper_total.csv', index=False)
save_avg_csv(merged_csv, '/root/work/tenset/scripts/pre_experiments/model_myself/result/([0c9a5ba46ffc5e1a9e5641018527117f,4,7,7,160,1,1,160,960,1,1,1,960,4,7,7,960],cuda).json/hyper/vae_extent_hyper_total.csv', top_k=1)